In [1]:
import os
import json
from collections import Counter, defaultdict
from pathlib import Path

In [2]:
FHIR_ROOT = Path(
    "/Users/angieyoon/Dropbox/Angie dropbox/UVA/DS6015Capstone/Forge-N-FHIR/synthetic-ivy-hip-emr/FHIR"
)

FHIR_ROOT.exists(), FHIR_ROOT.is_dir()

(True, True)

In [3]:
#how many files in there?
all_files = list(FHIR_ROOT.glob("*.json"))
len(all_files)

6029

In [4]:
# looking at one file
sample_file = all_files[0]

with open(sample_file) as f:
    data = json.load(f)

type(data), data.keys()

(dict,
 dict_keys(['resourceType', 'id', 'meta', 'status', 'class', 'type', 'subject', 'period', 'reasonCode', 'location']))

In [5]:
data.get("resourceType")

'Encounter'

In [6]:
#next file
sample_file = all_files[1]

with open(sample_file) as f:
    data = json.load(f)

type(data), data.keys()

(dict,
 dict_keys(['resourceType', 'id', 'meta', 'status', 'category', 'code', 'subject', 'encounter', 'effectiveDateTime', 'valueString']))

In [7]:
data.get("resourceType")

'Observation'

In [8]:
#count resource types
from collections import Counter

resource_counts = Counter()

for f in all_files:
    with open(f) as fh:
        obj = json.load(fh)
    resource_counts[obj.get("resourceType", "MISSING")] += 1

resource_counts

Counter({'Observation': 1895,
         'Encounter': 369,
         'FamilyMemberHistory': 369,
         'DocumentReference': 369,
         'ImagingStudy': 369,
         'EpisodeOfCare': 369,
         'MedicationRequest': 369,
         'Immunization': 369,
         'AllergyIntolerance': 369,
         'DiagnosticReport': 369,
         'Goal': 369,
         'Procedure': 368,
         'Patient': 76})

In [9]:
#look references
def refs(obj):
    out = {}
    for k, v in obj.items():
        if isinstance(v, dict) and "reference" in v:
            out[k] = v["reference"]
    return out

# check a few observations
for f in all_files[:5]:
    with open(f) as fh:
        o = json.load(fh)
    if o["resourceType"] == "Observation":
        print(refs(o))
        break

{'subject': 'Patient/patient-LA753', 'encounter': 'Encounter/encounter-2018_LA753-S6M'}


In [10]:
def ref_id(ref):
    """
    'Encounter/encounter-2018_LA753-S6M' -> 'encounter-2018_LA753-S6M'
    """
    if not ref or "/" not in ref:
        return None
    return ref.split("/", 1)[1]

In [11]:
ref_id("Patient/patient-LA753")

'patient-LA753'

In [12]:
#let's see encounter centric grouping will work
from collections import defaultdict

by_encounter = defaultdict(list)

for f in all_files:
    with open(f) as fh:
        obj = json.load(fh)

    enc = obj.get("encounter", {}).get("reference")
    if enc:
        by_encounter[ref_id(enc)].append(obj["resourceType"])

len(by_encounter)

369

In [13]:
#see one encounter
sample_enc = next(iter(by_encounter))
sample_enc, by_encounter[sample_enc]

('encounter-2018_LA753-S6M',
 ['Observation',
  'Observation',
  'Procedure',
  'Observation',
  'DiagnosticReport',
  'ImagingStudy',
  'Observation',
  'Observation',
  'Observation'])